In [1]:
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, time, datetime, timedelta
import os
from glob import glob
from scipy.integrate import trapz
import matplotlib.cm as cm
import locale
from dateutil.parser import *
from matplotlib.ticker import FuncFormatter, MaxNLocator
from functools import partial
import matplotlib.dates as mdates

In [2]:
figure1 = plt.figure()
ax1 = figure1.add_subplot(1, 1, 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
PATH = 'Potentiostate'
EXT = '*.DTA'
filterString = ''
all_filenames = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in sorted(glob(os.path.join(path, EXT)), key=os.path.getmtime)]
print(len(all_filenames))

59


In [4]:
skipRowsDict = {
    'LSV': 56,
    'CHRONOA': 61
}

all_files = []
for filename in all_filenames[0:50]:
    now = None
    identifier = None
    
    with open(filename, mode='rt') as file:
        lines = file.readlines()
        date = lines[3].strip().split('\t')[2]
        time = lines [4].strip().split('\t')[2]
        datetimeString = date + ' ' + time
        
        now = parse(datetimeString)
        
        identifier = lines[1].strip().split('\t')[1]
        

    data = pd.read_csv(filename, delim_whitespace=True, header=None, skiprows=skipRowsDict[identifier], names=['Pt', 'T', 'Vf', 'Im', 'Vu', 'Sig', 'Ach', 'IERange', 'Over', 'Cycle', 'Temp'], decimal=",")
    data.date = now
    data.identifier = identifier
    
    all_files.append(data)
print(len(all_files))


50


In [5]:
for z, file in enumerate(all_files):
    # file['intensity_smoothed'] = file['intensity'].rolling(200, win_type='nuttall').mean().shift(-100).fillna(1)
    # integrateValue = trapz(file['intensity'], x=file['wavelength'])
    file['T'] = pd.to_timedelta(file['T'], unit='s')
    file['actualTime'] = file.date + file['T']

In [6]:
ax1.clear()
ax1.set_ylim(-12E-6, +10e-6)
# ax1.set_xlim(-0.1, 0.9)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M:%S'))


# selectedFiles = all_files[1300:-1]
# selectedFiles = [all_files[i] for i in [0, 60, 100, 230]]


# selectedFiles = [all_files[i] for i in range(522, 895, 1)]    ######BEST SELECTION SO FAR, 5 CYCLES 

# selectedFiles = [all_files[i] for i in range(1, 895, 1)]

# selectedFiles = [all_files[i] for i in range(1, 1250, 1)]
selectedFiles = all_files

for z, file in enumerate(selectedFiles):
    ax1.plot(file['actualTime'], file['Im'])
    
    
figure1.autofmt_xdate()

In [11]:
lookupDatafile = pd.concat(selectedFiles)
print(lookupDatafile)

        Pt                      T        Vf            Im   Vu       Sig  Ach  \
0        0        0 days 00:00:01  3.115200  1.080170e-08  0.0  3.115220  0.0   
1        1        0 days 00:00:02  3.116250  3.588410e-09  0.0  3.114720  0.0   
2        2        0 days 00:00:03  3.115620  1.277510e-09  0.0  3.114220  0.0   
3        3        0 days 00:00:04  3.115050  4.978420e-10  0.0  3.113720  0.0   
4        4        0 days 00:00:05  3.114550 -2.523400e-09  0.0  3.113220  0.0   
...    ...                    ...       ...           ...  ...       ...  ...   
1995  1996        0 days 00:16:38  0.009682 -3.007780e-06  0.0  0.009994  0.0   
1996  1997 0 days 00:16:38.500000  0.009682 -3.007330e-06  0.0  0.009994  0.0   
1997  1998        0 days 00:16:39  0.009682 -3.007010e-06  0.0  0.009994  0.0   
1998  1999 0 days 00:16:39.500000  0.009683 -3.006570e-06  0.0  0.009994  0.0   
1999  2000        0 days 00:16:40  0.009683 -3.006150e-06  0.0  0.009994  0.0   

      IERange         Over 

In [35]:
figure2 = plt.figure()
ax2 = figure2.add_subplot(1, 1, 1)
ax2.plot(lookupDatafile['actualTime'], lookupDatafile['Vf'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
lookupTime = parse('Thu Dec 17 07:34:07 CET 2020')
lookupDatafile['Vf'][lookupDatafile['actualTime'] == lookupTime].mean()

0.10428